# Soil Health Prediction from Infrared Spectroscopy

This notebook demonstrates how to predict soil properties (Ca, P, pH, SOC, Sand) from mid-infrared spectral absorption measurements using machine learning.

## Problem Overview

- **Input**: 3,578 mid-infrared spectral features per sample
- **Output**: 5 continuous soil properties (multi-target regression)
- **Challenge**: High-dimensional data (more features than samples)
- **Solution**: Regularized models, spectral preprocessing, proper validation

## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Import our custom modules
import sys
sys.path.insert(0, '../src')

from preprocessing import preprocess_pipeline, snv_correction, savitzky_golay_filter
from utils import (
    load_afsis_data,
    standardize_targets,
    inverse_transform_targets,
    evaluate_predictions,
    print_evaluation_results
)
from models import (
    RidgeRegressionCV,
    PLSBaseline,
    RandomForestPCA,
    DeepLearningModel
)

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Set random seed
np.random.seed(42)

## 2. Load and Explore Data

Download the data from Kaggle:
https://www.kaggle.com/c/afsis-soil-properties/data

Place the CSV files in a `data/` directory.

In [ ]:
# Load data (update path as needed)
# X, y = load_afsis_data('../data/training.csv')

# For demonstration, create synthetic data
print("Creating synthetic data for demonstration...")
n_samples = 1000
n_features = 3578
n_targets = 5

X = np.random.randn(n_samples, n_features) * 0.1 + np.random.rand(n_samples, 1)
y = np.random.randn(n_samples, n_targets) * 10 + np.array([5000, 30, 6.5, 2.0, 50])

print(f"Data shape: X={X.shape}, y={y.shape}")
print(f"\nTarget statistics:")
target_names = ['Ca', 'P', 'pH', 'SOC', 'Sand']
stats_df = pd.DataFrame(y, columns=target_names).describe()
print(stats_df)

## 3. Visualize Spectral Data

In [ ]:
# Plot sample spectra
fig, ax = plt.subplots(figsize=(12, 4))

for i in range(5):
    ax.plot(X[i], alpha=0.7, label=f'Sample {i+1}')

ax.set_xlabel('Wavenumber Index')
ax.set_ylabel('Absorption')
ax.set_title('Sample Mid-Infrared Spectra')
ax.legend()
plt.tight_layout()
plt.show()

## 4. Data Splitting

Split data into training and validation sets. In practice, you should split by geographic sites to avoid data leakage.

In [ ]:
# Split data
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: X={X_train.shape}, y={y_train.shape}")
print(f"Validation set: X={X_val.shape}, y={y_val.shape}")

## 5. Spectral Preprocessing

Apply preprocessing to remove noise and standardize spectra:
- **SNV (Standard Normal Variate)**: Removes multiplicative scatter effects
- **Savitzky-Golay**: Smooths spectra while preserving features

In [ ]:
# Preprocess spectral data
X_train_prep = preprocess_pipeline(X_train, methods=['snv', 'savgol'])
X_val_prep = preprocess_pipeline(X_val, methods=['snv', 'savgol'])

# Visualize preprocessing effect
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(X_train[0], alpha=0.7, label='Raw')
axes[0].set_title('Raw Spectrum')
axes[0].set_xlabel('Wavenumber Index')
axes[0].set_ylabel('Absorption')

axes[1].plot(X_train_prep[0], alpha=0.7, label='Preprocessed', color='orange')
axes[1].set_title('Preprocessed Spectrum (SNV + Savitzky-Golay)')
axes[1].set_xlabel('Wavenumber Index')
axes[1].set_ylabel('Absorption')

plt.tight_layout()
plt.show()

## 6. Target Standardization

Standardize each target independently since they have vastly different scales (pH: 4-9 vs Ca: 0-40,000 ppm).

In [ ]:
# Standardize targets
y_train_scaled, y_val_scaled, scalers = standardize_targets(y_train, y_val)

print("Target standardization complete.")
print(f"Scaled training targets: mean={y_train_scaled.mean(axis=0)}, std={y_train_scaled.std(axis=0)}")

## 7. Baseline Models

### 7.1 Ridge Regression with Cross-Validation

In [ ]:
# Train Ridge Regression
print("Training Ridge Regression with CV...")
ridge_model = RidgeRegressionCV(cv=5)
ridge_model.fit(X_train_prep, y_train_scaled)

# Make predictions
y_pred_ridge_scaled = ridge_model.predict(X_val_prep)
y_pred_ridge = inverse_transform_targets(y_pred_ridge_scaled, scalers)

# Evaluate
print("\nRidge Regression Results:")
ridge_results = evaluate_predictions(y_val, y_pred_ridge, target_names)
print_evaluation_results(ridge_results)

### 7.2 Partial Least Squares (PLS)

In [ ]:
# Train PLS
print("Training PLS...")
pls_model = PLSBaseline(n_components=50)
pls_model.fit(X_train_prep, y_train_scaled)

# Make predictions
y_pred_pls_scaled = pls_model.predict(X_val_prep)
y_pred_pls = inverse_transform_targets(y_pred_pls_scaled, scalers)

# Evaluate
print("\nPLS Results:")
pls_results = evaluate_predictions(y_val, y_pred_pls, target_names)
print_evaluation_results(pls_results)

### 7.3 Random Forest with PCA

In [ ]:
# Train Random Forest
print("Training Random Forest with PCA...")
rf_model = RandomForestPCA(n_components=50, n_estimators=100)
rf_model.fit(X_train_prep, y_train_scaled)

# Make predictions
y_pred_rf_scaled = rf_model.predict(X_val_prep)
y_pred_rf = inverse_transform_targets(y_pred_rf_scaled, scalers)

# Evaluate
print("\nRandom Forest Results:")
rf_results = evaluate_predictions(y_val, y_pred_rf, target_names)
print_evaluation_results(rf_results)

## 8. Advanced Models

### 8.1 1D Convolutional Neural Network

In [ ]:
# Train 1D CNN
print("Training 1D CNN...")
cnn_model = DeepLearningModel(
    model_type='conv1d',
    input_size=X_train_prep.shape[1],
    n_targets=y_train_scaled.shape[1],
    epochs=50,  # Use fewer epochs for demo
    batch_size=32
)

cnn_model.fit(X_train_prep, y_train_scaled, X_val_prep, y_val_scaled)

# Make predictions
y_pred_cnn_scaled = cnn_model.predict(X_val_prep)
y_pred_cnn = inverse_transform_targets(y_pred_cnn_scaled, scalers)

# Evaluate
print("\n1D CNN Results:")
cnn_results = evaluate_predictions(y_val, y_pred_cnn, target_names)
print_evaluation_results(cnn_results)

### 8.2 Multi-Task Neural Network

In [ ]:
# Train Multi-Task Network
print("Training Multi-Task Network...")
mt_model = DeepLearningModel(
    model_type='multitask',
    input_size=X_train_prep.shape[1],
    n_targets=y_train_scaled.shape[1],
    epochs=50,  # Use fewer epochs for demo
    batch_size=32
)

mt_model.fit(X_train_prep, y_train_scaled, X_val_prep, y_val_scaled)

# Make predictions
y_pred_mt_scaled = mt_model.predict(X_val_prep)
y_pred_mt = inverse_transform_targets(y_pred_mt_scaled, scalers)

# Evaluate
print("\nMulti-Task Network Results:")
mt_results = evaluate_predictions(y_val, y_pred_mt, target_names)
print_evaluation_results(mt_results)

## 9. Compare Models

In [ ]:
# Collect all results
all_results = {
    'Ridge': ridge_results,
    'PLS': pls_results,
    'Random Forest': rf_results,
    '1D CNN': cnn_results,
    'Multi-Task': mt_results
}

# Create comparison DataFrame
comparison_data = []
for model_name, results in all_results.items():
    for target in target_names:
        comparison_data.append({
            'Model': model_name,
            'Target': target,
            'RMSE': results[target]['RMSE'],
            'RPD': results[target]['RPD']
        })

comparison_df = pd.DataFrame(comparison_data)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# RMSE comparison
pivot_rmse = comparison_df.pivot(index='Model', columns='Target', values='RMSE')
pivot_rmse.plot(kind='bar', ax=axes[0], width=0.8)
axes[0].set_title('RMSE Comparison by Target')
axes[0].set_ylabel('RMSE')
axes[0].legend(title='Target', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].tick_params(axis='x', rotation=45)

# RPD comparison
pivot_rpd = comparison_df.pivot(index='Model', columns='Target', values='RPD')
pivot_rpd.plot(kind='bar', ax=axes[1], width=0.8)
axes[1].set_title('RPD Comparison by Target')
axes[1].set_ylabel('RPD (higher is better)')
axes[1].axhline(y=2.0, color='r', linestyle='--', label='Good (RPD=2.0)')
axes[1].axhline(y=3.0, color='g', linestyle='--', label='Excellent (RPD=3.0)')
axes[1].legend(title='Target', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\nModel Rankings by Mean RPD:")
mean_rpd = {}
for model_name, results in all_results.items():
    mean_rpd[model_name] = results['Mean']['RPD']

ranking_df = pd.DataFrame.from_dict(mean_rpd, orient='index', columns=['Mean RPD'])
ranking_df = ranking_df.sort_values('Mean RPD', ascending=False)
print(ranking_df)

## 10. Prediction Scatter Plots

Visualize predictions vs. actual values for the best model.

In [ ]:
# Use Ridge predictions for visualization
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, target in enumerate(target_names):
    ax = axes[i]
    
    # Scatter plot
    ax.scatter(y_val[:, i], y_pred_ridge[:, i], alpha=0.5)
    
    # Perfect prediction line
    min_val = min(y_val[:, i].min(), y_pred_ridge[:, i].min())
    max_val = max(y_val[:, i].max(), y_pred_ridge[:, i].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
    
    ax.set_xlabel(f'Actual {target}')
    ax.set_ylabel(f'Predicted {target}')
    ax.set_title(f'{target} (R² = {ridge_results[target]["R2"]:.3f})')
    ax.grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

## 11. Summary and Next Steps

### Key Takeaways:

1. **Preprocessing is critical**: SNV and Savitzky-Golay smoothing improve model performance
2. **Regularization is essential**: Ridge regression works well for high-dimensional data
3. **PLS is effective**: Standard chemometrics baseline performs competitively
4. **Deep learning potential**: CNNs can capture local spectral patterns
5. **Multi-task learning**: Joint prediction can leverage correlations between soil properties

### Next Steps:

1. **Download real data**: Use the Africa Soil Property Prediction dataset from Kaggle
2. **Geographic splitting**: Split by site IDs to avoid data leakage
3. **Hyperparameter tuning**: Optimize model parameters with cross-validation
4. **Ensemble methods**: Combine multiple models for better predictions
5. **Feature engineering**: Try different preprocessing methods and combinations
6. **Scale up**: Use the iSDA AfSIS Full Release with 50,000+ samples

### Resources:

- Kaggle Competition: https://www.kaggle.com/c/afsis-soil-properties
- iSDA AfSIS Data: https://www.isda-africa.com/
- Documentation: See README.md for more details